In [1]:
# reference data gathering 

import csv
import json
import pandas as pd
import re
import logging
import os
import random
import sys
from heapq import nlargest
import openai
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd


with open('../../data_set_v1.0/author_json.json', 'r',encoding='utf-8') as f:
    authors = json.load(f)
print(len(authors))

with open('../../data_set_v1.0/paper_json.json', 'r',encoding='utf-8') as f:
    papers = json.load(f)
print(len(papers))
# sampled papers to 500 
count = 0
max_paper = 500
author_paper_count = []
paper_author_count = []
papers_sampled = {}
authors_sampled = {}
for i in papers:
    if count < 500:
        flag = True
        for author in papers[i]["authors"]:
            if len(authors[author]["papers"])<5:
                flag = False
        if flag and len(papers[i]["authors"]) > 1:
            paper_author_count.append(len(papers[i]["authors"]))
            papers_sampled[i] = papers[i]
            papers_sampled[i]["author_history"] = {}
            
            for author in papers[i]["authors"]:
                papers_sampled[i]["author_history"][author] = {}
                papers_sampled[i]["author_history"][author]["authors_history_text"] = [papers[i]["abstract"]]
                papers_sampled[i]["author_history"][author]["authors_history_dict"] = [{"Abstract":"","Title":""}]
                paper_count_tmp = 0
                for paper_tmp in authors[author]["papers"]:
                    if paper_count_tmp < max_paper and paper_tmp["title"] != papers[i]["title"]:
                        papers_sampled[i]["author_history"][author]["authors_history_text"].append("Abstract: "+paper_tmp["abstract"]+"\nTitle: "+paper_tmp["title"])
                        papers_sampled[i]["author_history"][author]["authors_history_dict"].append({"Abstract":paper_tmp["abstract"],"Title":paper_tmp["title"]})
                        paper_count_tmp = paper_count_tmp + 1
                author_paper_count.append(len(authors[author]["papers"]))
                authors_sampled[author] = authors[author]
            count = count + 1
print("papers count after sampled: ",len(papers_sampled))
print("authors count after sampled: ",len(authors_sampled))


7931
3275
papers count after sampled:  500
authors count after sampled:  1080


In [3]:
# Statistics 
for i in papers_sampled:
    for j in papers_sampled[i]:
        print(j)
    break
# author_cnt = [len(papers_sampled[i]["abstract"]) for i in papers_sampled]
# print(sum(author_cnt)/len(author_cnt))

paperId
title
abstract
year
authors
author_history


In [2]:
#!/usr/bin/env python3


import argparse
import csv
import os
from requests import Session
from typing import Generator, TypeVar
import requests

S2_API_KEY = "mEwrkvDmRc9KFXu76cAAk16KJxHNNUuf9tYaot1g"


def get_paper_batch( ids, fields):
    params = {
        'fields': fields
    }
    headers = {
        'X-API-KEY': S2_API_KEY,
    }
    body = {
        'ids': ids,
    }

    # https://api.semanticscholar.org/api-docs/graph#tag/Paper-Data/operation/post_graph_get_papers
    with requests.post('https://api.semanticscholar.org/graph/v1/paper/batch',
                       params=params,
                       headers=headers,
                       json=body
                       ) as response:
        return response.content
paper2pdfs = {}
paperIds = [i for i in papers_sampled]
paper2pdfs = get_paper_batch(paperIds,["openAccessPdf"])
paper2pdfs = json.loads(paper2pdfs)

# print(len(paper2pdfs))
id2pdfs = {}
for paper in paper2pdfs:
    # print(paper)
    if paper is not None and "openAccessPdf" in paper:
        if paper["openAccessPdf"] is not None:
            id2pdfs[paper["paperId"]] = paper["openAccessPdf"]
    
len(id2pdfs)

163

In [40]:
import requests
requests.timeout = 20
count = 0
for i in id2pdfs:
    if "url" in id2pdfs[i]:
        try:
            url = id2pdfs[i]["url"]
            # url = "https://link.springer.com/content/pdf/bfm:978-3-031-02549-5/1"
            myfile = requests.get(url, allow_redirects=True)
            filename = i +".pdf"
            open(filename, 'wb').write(myfile.content)
            print(str(count)+" done!")
            count = count + 1
        except Exception as e:
            continue
    # break

0 done!
1 done!
2 done!
3 done!
4 done!
5 done!
6 done!
7 done!
8 done!
9 done!
10 done!
11 done!
12 done!
13 done!
14 done!
15 done!
16 done!
17 done!
18 done!
19 done!
20 done!
21 done!
22 done!
23 done!
24 done!
25 done!
26 done!
27 done!
28 done!
29 done!
30 done!
31 done!
32 done!
33 done!
34 done!
35 done!
36 done!
37 done!
38 done!
39 done!
40 done!
41 done!
42 done!
43 done!
44 done!
45 done!
46 done!
47 done!
48 done!
49 done!
50 done!
51 done!
52 done!
53 done!
54 done!
55 done!
56 done!
57 done!
58 done!
59 done!
60 done!
61 done!
62 done!
63 done!
64 done!
65 done!
66 done!
67 done!
68 done!
69 done!
70 done!
71 done!
72 done!
73 done!
74 done!
75 done!
76 done!
77 done!
78 done!
79 done!
80 done!
81 done!
82 done!
83 done!
84 done!
85 done!
86 done!
87 done!
88 done!
89 done!
90 done!
91 done!
92 done!
93 done!
94 done!
95 done!
96 done!
97 done!
98 done!
99 done!
100 done!
101 done!
102 done!
103 done!
104 done!
105 done!
106 done!
107 done!
108 done!
109 done!
110 done!


In [4]:
import os
for i in id2pdfs:
    if "url" in id2pdfs[i]:
        os.system('/anaconda/envs/doc2json/bin/python ../../../s2orc-doc2json/doc2json/grobid2json/process_pdf.py -i '+i+'.pdf -t temp_dir/ -o ./paperJson/')


runtime: 0.686 seconds 
done.
runtime: 0.247 seconds 
done.
runtime: 0.166 seconds 
done.
runtime: 0.426 seconds 
done.
runtime: 0.38 seconds 
done.
2f9a1286e7af4ab7706ad8cfcc8c8742a1964939.pdf
runtime: 3.332 seconds 
done.
runtime: 0.199 seconds 
done.
e708fe09fcbcebba443db826c6f0d6c839137cb0.pdf
Processing failed with error 500
runtime: 0.238 seconds 
done.
runtime: 0.194 seconds 
done.
runtime: 0.478 seconds 
done.
runtime: 0.358 seconds 
done.
1c5b80259a8b8b4d4f286a86ab2d5ea983ee0db5.pdf
Processing failed with error 500
runtime: 0.212 seconds 
done.
runtime: 0.153 seconds 
done.
runtime: 0.209 seconds 
done.
runtime: 0.243 seconds 
done.
runtime: 0.25 seconds 
done.
runtime: 0.168 seconds 
done.
runtime: 0.121 seconds 
done.
runtime: 0.278 seconds 
done.
286c57dd231dcac718f556f76ece89db7571d380.pdf
Processing failed with error 500
6b9f883c89e0c350d4cc9c0d5aa39c75afe252ae.pdf
runtime: 3.057 seconds 
done.
d5094bfeff961a09318c9506395e1f877686b85a.pdf
Processing failed with error 500


Traceback (most recent call last):
  File "../../../s2orc-doc2json/doc2json/grobid2json/process_pdf.py", line 100, in <module>
    process_pdf_file(input_path, temp_path, output_path)
  File "../../../s2orc-doc2json/doc2json/grobid2json/process_pdf.py", line 71, in process_pdf_file
    assert os.path.exists(tei_file)
AssertionError
Traceback (most recent call last):
  File "../../../s2orc-doc2json/doc2json/grobid2json/process_pdf.py", line 100, in <module>
    process_pdf_file(input_path, temp_path, output_path)
  File "../../../s2orc-doc2json/doc2json/grobid2json/process_pdf.py", line 60, in process_pdf_file
    raise FileNotFoundError(f"{input_file} doesn't exist")
FileNotFoundError: 7bcfe45fbdccbfc4667540b3c54b4aff398d140c.pdf doesn't exist
Traceback (most recent call last):
  File "../../../s2orc-doc2json/doc2json/grobid2json/process_pdf.py", line 100, in <module>
    process_pdf_file(input_path, temp_path, output_path)
  File "../../../s2orc-doc2json/doc2json/grobid2json/process_p

In [4]:
for i in id2pdfs:
    if os.path.exists("paperJson/"+i+".json"):
        print(i)


In [6]:
id2introduction = {}
count = 0

for i in id2pdfs:
    if os.path.exists("../paperJson/"+i+".json"):
        # print(i)
        if "url" in id2pdfs[i]:
            with open('../paperJson/'+i+'.json', 'r',encoding='utf-8') as f:
                paper = json.load(f)
            SectionDict = {}
            introductionSecName = ""
            introductions = []
            for j in paper["pdf_parse"]["body_text"]:
                sec = j["section"].lower()
                if "introduction" in sec:
                    introductionSecName = j["section"]
                    break
            if len(introductionSecName)> 0:
                for j in paper["pdf_parse"]["body_text"]:
                    if j["section"] == introductionSecName:
                        introductions.append(j["text"])
                id2introduction[i] = " ".join(introductions)
                count = count + 1
            # print(paper)
            # break
print(len(id2introduction),count)


130862d54894966552cb85d3ee6f739f885d4989
e28bdc373de80d7ec0e64631a89e64fbdcdae230
fb2bb5777f1b1bd745070c006265edf8feb5f29f
51b502b9ce774a615474ed8629e74d0dfaa33ee3
6f58d8b98c652897842afdd023c535f9724b4eb2
8fe2f59ff3733f9ee50ffa295beda502f4e268e2
f9a9f3f016fa3123c3059cca66314d26f2357155
819f36c0ddae12132d60ebc5cc0a19f7db8668b1
564614da76b5d9020c700b78e1fe154bd590c47d
fac3a6f272428e9d6879227ef76c1fa9397b317f
2f9a1286e7af4ab7706ad8cfcc8c8742a1964939
188e778ae2b5c1dee1dd76ce9c786f566f7e7079
717a8985cc3fcbd285efe50dbcb8fe5bb12921b1
09402e20ca49842e1d1d967d3c5a530e8167b9b1
22cf911f7111f46fd3eeb75d95c01ae10187ad8d
1112e2422dbb90df87da2df6afc719e90f5e0af6
0ed846e87ce0961d162e9115b4e9837537138e3a
cf20d6130160843e5cd8ef8f5553d66a920bc6f3
d48a7458182aeebf080d4bc2b0100e6d126c1b79
35e06978cd069786ff7ab05ac29d6e2e28f1bad6
05882b3376dddab5066015b46704bb7bfe3946cc
87cc9fb5129781b74750d88f83472f2cd644ca60
59d3ec40b4f17ed94dc5ae510c316ac511915031
3713667bb3ce322c3bb078f4499017697a07282e
761bf03e086ce887

In [14]:
paper_updated = {}
author_updated = {}
authorDict = {}

for i in id2introduction:
    if i in papers_sampled:
        
        paper = papers_sampled[i]
        paper["introduction"] = id2introduction[i]
        paper_updated[i] = paper
        for authorId in papers_sampled[i]["authors"]:
            authorDict[authorId] = 1
print(len(authorDict)) 
for i in authorDict:
    if i not in authors_sampled:
        author_updated[i] = authors_sampled[i]
# paper

# # id2introduction
with open('../../data_set_v3.0/paper_json.json', 'w', encoding='utf-8') as f:
    json.dump(paper_updated, f, indent=4)
with open('../../data_set_v3.0/author_json.json', 'w', encoding='utf-8') as f:
    json.dump(author_updated, f, indent=4)

# papers_sampled["188e778ae2b5c1dee1dd76ce9c786f566f7e7079"]

281


In [10]:
input_history = pd.read_csv('../naive_method_v1.0/paper_to_author_cot_5_examples_.tsv', sep='\t', header=None, names=["paper_index","author_index","example_index","result","abstract","title"])
user_keywords_topics = pd.read_csv('../naive_method_v1.0/author_profiling_20_examples_.tsv', sep='\t', header=None, names=["author_index","keywords","topics"])
user_interests = pd.read_csv('../task1/author_interests_gold.tsv', sep='\t', header=None, names=["author_index","interests"])
id2authorTopics = {}
id2authorInterests = {}
id2ICLexamples = {}
for i in range(len(user_keywords_topics["author_index"])):
    id2authorTopics[user_keywords_topics["author_index"][i]] = {"keywords":user_keywords_topics["keywords"][i],"topics":user_keywords_topics["topics"][i]}

for i in range(len(user_interests["author_index"])):
    id2authorInterests[user_interests["author_index"][i]] = {"interests":user_interests["interests"][i]}

for i in range(len(input_history["paper_index"])):
    paperId = input_history["paper_index"][i]
    authorId = input_history["author_index"][i]
    if paperId not in id2ICLexamples:
        id2ICLexamples[paperId] = {}
    if authorId not in id2ICLexamples[paperId]:
        id2ICLexamples[paperId][authorId]=[]
    id2ICLexamples[paperId][authorId].append({"abstract":input_history["abstract"][i],"title":input_history["title"][i],"result":input_history["result"][i]})

# id2ICLexamples

In [ ]:
# Task-2 Zero-shot
error_list=[]
i = 0
openai.api_key = "31b8638c9eea48709a596501490f9e88"
openai.api_base = "https://gersteinbiocodeeval-eastus2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/"),
    ('31b8638c9eea48709a596501490f9e88', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/")
]

    # prompt_construction v1.6
def construct_prompt_zero_shot_1(reference_dict,paperId):
    ref_example = []
    for index,item in enumerate(reference_dict):
        if index <50:
            ref_example.append("Reference Paper #"+str(index)+"\n"+"Title: "+item["title"])
            
    prompt_str = ""

    prompt_str = prompt_str + "You're a researcher working on a new paper.\n"
    prompt_str = prompt_str + "Now you have gathered some related science papers for reference, each of them contains a title.\n"
    prompt_str = prompt_str + "\n".join(ref_example) + "\n"
    prompt_str = prompt_str + "Please fully consider the above reference papers and brainstorm one potential title for a new scientific work:\n"
    prompt_str = prompt_str + "Title: "

    return prompt_str

def construct_prompt_few_shot(reference_dict,paperId,authorDicts):
    prompt_str = ""
    keywords = ""
    interests = ""
    authorId = ""
    topics = ""
    authorDict = []
    ref_example = []
    history_examples = []
    for i in authorDicts:
        authorId = i
        authorDict = authorDicts[authorId]
        break

    for index,item in enumerate(reference_dict):
        if index <50:
            ref_example.append("Reference Paper #"+str(index)+"\n"+"Title: "+item["title"])
    if authorId in id2authorInterests:
        if "interests" in id2authorInterests[authorId]:
            interests =  id2authorInterests[authorId]["interests"]
    if "keywords" in id2authorTopics[authorId]:
        keywords = str(id2authorTopics[authorId]["keywords"])
    if "topics" in id2authorTopics[authorId]:
        topics = str(id2authorTopics[authorId]["topics"])
    for index,item in enumerate(authorDict):
        if index <5:
            # history_examples.append("Example #"+str(index)+"\nAbstract: "+item["abstract"]+"\nStyle: "+item["result"]+"\nTitle: "+item["title"])
            history_examples.append("History Paper #"+str(index)+": "+item["title"])

    
    prompt_str = prompt_str + "Assuming you are an expert researcher in these areas:\n"
    prompt_str = prompt_str + " ".join(interests)
    prompt_str = prompt_str + "Here're some titles and paper abstracts that you've written.\n"
    prompt_str = prompt_str + "\n".join(history_examples) + "\n"


    prompt_str = prompt_str + "Now you're working on a new paper.\n"
    prompt_str = prompt_str + "You have gathered some related science papers for reference, each of them contains a title.\n"
    prompt_str = prompt_str + "\n".join(ref_example) + "\n"

    prompt_str = prompt_str + "Please fully consider the above reference papers and your previous works and brainstorm one most promising title for a science work:\n"
    prompt_str = prompt_str + "Title: "

    return prompt_str


def construct_prompt_few_shot_for_multi_agent_input(reference_dict,paperId,authorId,authorDict):
    prompt_str = ""
    keywords = ""
    interests = ""
    topics = ""
    ref_example = []
    history_examples = []

    for index,item in enumerate(reference_dict):
        if index <50:
            ref_example.append("Reference Paper #"+str(index)+"\n"+"Title: "+item["title"])
    if authorId in id2authorInterests:
        if "interests" in id2authorInterests[authorId]:
            interests =  id2authorInterests[authorId]["interests"]
    for index,item in enumerate(authorDict):
        if index <5:
            # history_examples.append("Example #"+str(index)+"\nAbstract: "+item["abstract"]+"\nStyle: "+item["result"]+"\nTitle: "+item["title"])
            history_examples.append("History Paper #"+str(index)+": "+item["title"])

    
    prompt_str = prompt_str + "Assuming you are an expert researcher in these areas:\n"
    prompt_str = prompt_str + " ".join(interests)
    prompt_str = prompt_str + "Here're some titles and paper abstracts that you've written.\n"
    prompt_str = prompt_str + "\n".join(history_examples) + "\n"


    prompt_str = prompt_str + "Now you're working on a new paper.\n"
    prompt_str = prompt_str + "You have gathered some related science papers for reference, each of them contains a title.\n"
    prompt_str = prompt_str + "\n".join(ref_example) + "\n"

    prompt_str = prompt_str + "Please fully consider the above reference papers and your previous works and brainstorm one most promising title for a science work:\n"
    prompt_str = prompt_str + "Title: "

    return prompt_str

def run_openai(prompt,key_bundles,i,indice):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ')
        # print(indice+"\t"+result+"\n")
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
            # print(indice+"\t"+result+"\n")
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                # print(indice+"\t"+result+"\n")
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i)
        #print(result)
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i)
        #print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

id2result = {}
with open("few_shot_1_title_single_author_for_multi_agents_input.tsv", "wb", buffering=0) as out_file:
    # for indice,paperId in enumerate(id2ICLexamples):
    #     # prompt = construct_prompt_zero_shot_1(id2citations[paperId],paperId)
    #     if paperId in id2citations:
    #         prompt = construct_prompt_few_shot(id2citations[paperId],paperId,id2ICLexamples[paperId])
    #         # print(prompt)
    #         # break
    #         result = run_openai(prompt,key_bundles,i,indice)
    #         print(indice, result)
    #         id2result[paperId] = result
    #         write_str = bytes(str(paperId)+"\t"+result+"\n", 'utf-8')
    #         out_file.write(write_str)
    for indice,paperId in enumerate(id2ICLexamples):
        # prompt = construct_prompt_zero_shot_1(id2citations[paperId],paperId)
        if paperId in id2citations:
            id2result[paperId] = []
            for authorId in id2ICLexamples[paperId]:
                prompt = construct_prompt_few_shot_for_multi_agent_input(id2citations[paperId],paperId,authorId,id2ICLexamples[paperId][authorId])
                # print(prompt)
                # break
                result = run_openai(prompt,key_bundles,i,indice)
                print(indice, result)
                id2result[paperId].append({authorId:result})
                write_str = bytes(str(paperId)+"\t"+str(authorId)+"\t"+result+"\n", 'utf-8')
                out_file.write(write_str)

In [68]:
author_roles = pd.read_csv('../naive_method_v1.0/paper_author_roles.tsv', sep='\t', header=None, names=["paper_index","author_index","role"])

id2authorRole = {}
for i in range(len(author_roles["paper_index"])):
    paperId = author_roles["paper_index"][i]
    authorId = author_roles["author_index"][i]
    if paperId not in id2authorRole:
        id2authorRole[paperId] = {}
    id2authorRole[paperId][authorId] = author_roles["role"][i]
id2authorRole
error_list=[]
i = 0
openai.api_key = "31b8638c9eea48709a596501490f9e88"
openai.api_base = "https://gersteinbiocodeeval-eastus2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/"),
    ('31b8638c9eea48709a596501490f9e88', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/")
]

    # prompt_construction v1.6
def construct_prompt_cot_agg(reference_dict,paperId,authorTitleList):
    prompt_str = ""
    authors_list = []
    
    ref_example = []

    for index,item in enumerate(reference_dict):
        if index <50:
            ref_example.append("Reference Paper #"+str(index)+"\n"+"Title: "+item["title"])


    for index,item in enumerate(authorTitleList):
        authorId = next(iter(item))
        # authors_list.append("Author #"+str(index)+"\nTitle: "+author_dict[item]["title"])
        authors_list.append("Author #"+str(index)+"\nTitle: "+item[authorId]+"\nRole in research team: "+id2authorRole[paperId][authorId])

    question = papers_sampled[paperId]["abstract"]
    prompt_str = ""
    prompt_str = prompt_str + "Assuming you’re the leading author of a research team and you’re working on a new paper:\n"
    prompt_str = prompt_str + "Now You have gathered some related science papers for reference, each of them contains a title.\n"
    prompt_str = prompt_str + "\n".join(ref_example) + "\n"

    prompt_str = prompt_str + "Authors from your research team have proposed the following titles based on their expertises.\n"
    prompt_str = prompt_str + "\n".join(authors_list) + "\n"

    prompt_str = prompt_str + "Please fully consider the above reference papers and the above proposed title comprehensively and brainstorm one most promising title for a science work, keeping it short and faithful:\n"
    prompt_str = prompt_str + "Title: "
    # prompt_str = prompt_str + "Please consider the above proposed title comprehensively and write a title for the given abstract, keeping it short and faithful:\n"
    # prompt_str = prompt_str + "Title: "

    return prompt_str


id2result_multi_agent = {}
with open("multi_agent_result_method_3.tsv", "wb", buffering=0) as out_file:
# with open("naive_method_v1.0/zero_shot_results.tsv", "wb", buffering=0) as out_file:
    for indice,paperId in enumerate(id2result):
        if paperId in id2citations:
            prompt = construct_prompt_cot_agg(id2citations[paperId],paperId,id2result[paperId])
            # print(prompt)
            # break
            result = run_openai(prompt,key_bundles,i,indice)
            print(indice, result)
            id2result_multi_agent[paperId] = result
            write_str = bytes(str(paperId)+"\t"+result+"\n", 'utf-8')
            out_file.write(write_str)


0 Model Transformation Evolution: Best Practices and Framework for Industry Adoption.
1 Systematic Review of Software Engineering Research: Insights and Guidelines for Evidence-Based Decision Making and Experimental Design.
2 Smart Contract Testing and Analysis: Challenges, Techniques, and Security Considerations
3 Integrating Crowdsourcing and Agile Methods in Software Engineering: A Multi-Method Study on Process Theories and Taxonomies.
4 Sentiment Analysis in Software Engineering: Challenges, Opportunities, and Future Directions
5 "Integrating Best Practices for Machine Learning and Software Engineering: A Comprehensive Study"
6 App Store Analysis for Software Engineering: A Comprehensive Study of Reviews, Ratings, and Updates.
7 "Grounded Theory in Software Engineering: Exploring Empirical Research Methods and Theoretical Frameworks"
8 Emotional States in Software Engineering: A Comprehensive Survey of Sentiment Analysis Techniques and Applications.
9 Improving Bug Localization in 

In [13]:
results = pd.read_csv('multi_agent_result_method_3.tsv', sep='\t', header=None, names=["paper_index","result"])
id2result_multi_agent = {}
for i in range(len(results["result"])):
    id2result_multi_agent[results["paper_index"][i]] = results["result"][i]

from rouge import Rouge
import evaluate
def postprocess_text_generation(preds, labels):
    preds = [pred.replace("\"","").strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def create_metric_rouge():
    rouge_metric = evaluate.load('rouge')
    def compute_metrics(decoded_preds, decoded_labels):
        decoded_preds, decoded_labels = postprocess_text_generation(decoded_preds, decoded_labels)
        result_rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"rouge-1" : result_rouge["rouge1"], "rouge-L" : result_rouge["rougeL"]}
        return result
    return compute_metrics
pred = []
gold = []
for i in id2citations:
    if i in papers_sampled and i in id2result_multi_agent:
        pred.append(id2result_multi_agent[i])
        gold.append(papers_sampled[i]["title"])
# print(pred)
metric = create_metric_rouge()
print(metric(pred, gold))
# results

2023-10-07 09:39:22.928451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-07 09:39:43.802358: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-07 09:39:43.802509: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-10-07 09:39:43.802523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

{'rouge-1': 0.3329395614080458, 'rouge-L': 0.27680985187196927}


In [ ]:
error_list=[]
i = 0
openai.api_key = "0de53d4ada864cd18e3771d16551aee1"
openai.api_base = "https://biocodeeval-openai.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('0de53d4ada864cd18e3771d16551aee1', "https://biocodeeval-openai.openai.azure.com/"),
    ('0de53d4ada864cd18e3771d16551aee1', "https://biocodeeval-openai.openai.azure.com/"),
    ('0de53d4ada864cd18e3771d16551aee1', "https://biocodeeval-openai.openai.azure.com/")
]


def run_openai(prompt,key_bundles,i,indice):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ')
        # print(indice+"\t"+result+"\n")
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
            # print(indice+"\t"+result+"\n")
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                # print(indice+"\t"+result+"\n")
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i)
        #print(result)
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i)
        #print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

# G-EVAL 
def construct_prompt_g_eval(source,output,refs,dimension):

    Name = ""
    refs_texts = []
    Description = ""
    Instruction = "Evaluation Steps:\n"
    dimensions_single_inputs = ["novelty","informativeness"]
    ICL = "Example #1: \nReference:\nRef #1: Focus Groups: Supporting Effective Product Development\nRef #2: Basics of Software Engineering Experimentation\nRef #3: Focus Groups: Supporting Effective Product Development\nRef #4: The Focus Group Interview: A Method for Assessing Users' Evaluation of Library Service\nRef #5: The role of experimentation in software engineering: past, current, and future\nPredicted title: Using the focus group method in software engineering: obtaining practitioner and user experiences\nNovelty score: 3\n\n\nExample #2: \nReference:\nRef #1: Research in software engineering: an analysis of the literature\nRef #2: An approach to optimizing software development team size\nRef #3: A systematic review of Web engineering research\nRef #4: Automatic generation of basic behavior schemas from UML class diagrams\nRef #5: Model Transformation: The Heart and Soul of Model-Driven Software Development\nPredicted title: Systematic Mapping Studies in Software Engineering\nNovelty score: 2\n\nExample #3: \nReference:\nRef #1: Qualitative research methods in Software Engineering\nRef #2: Theory-oriented software engineering\nRef #3: Using a grounded theory approach for exploring software product management challenges\nRef #4: Grounded Theory for Qualitative Research: A Practical Guide\nRef #5: Grounded theory for geeks\nPredicted title: Grounded Theory in Software Engineering Research\nNovelty score: 1\n"
    count = 0
    for ref in refs:
        if count<5:
            ref_text = ""
            ref_text = ref_text + "Ref #"+str(count) +": " + ref["title"]
            refs_texts.append(ref_text)
            count = count + 1

    if dimension == "coherence":
        Name = "Coherence"
        Description = "Coherence (1-5) - the collective quality of all sentences. We align this dimension with the DUC quality question of structure and coherence whereby \"the title should be well-structured and well-organized. The title should not just be a heap of related information, but should build from sentence to a coherent body of information about a topic.\"\n"
        Instruction = Instruction + "1. Read the science paper abstract carefully and identify the main topic and key points." + "\n"
        Instruction = Instruction + "2. Read the title and compare it to the science paper abstract. Check if the title covers the main topic and key points of the paper abstract, and if it presents them in a clear and logical order." + "\n"
        Instruction = Instruction + "3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria." + "\n"
    elif dimension == "consistency":
        Name = "Consistency"
        Description = "Consistency (1-5) - the factual alignment between the title and the science paper abstract. A factually consistent title contains only statements that are entailed by the source document. Annotators were also asked to penalize titles that contained hallucinated facts. \n"
        Instruction = Instruction + "1. Read the science paper abstract carefully and identify the main facts and details it presents." + "\n"
        Instruction = Instruction + "2. Read the title and compare it to the source abstract. Check if the title contains any factual errors that are not supported by the abstract." + "\n"
        Instruction = Instruction + "3. Assign a score for consistency on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria." + "\n"
    elif dimension == "fluency":
        Name = "Fluency (1-3)"
        Description = "Fluency (1-3): the quality of the title in terms of grammar, spelling, punctuation, word choice, and sentence structure.\n"
        Description = Description + "- 1: Poor. The title has many errors that make it hard to understand or sound unnatural.\n"
        Description = Description + "- 2: Fair. The title has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.\n"
        Description = Description + "- 3: Good. The title has few or no errors and is easy to read and follow.\n"
        Instruction = ""
    elif dimension == "relevance":
        Name = "Relevance"
        Description = "Relevance (1-5) - selection of important content from the source. The title should include only important information from the source document. Annotators were instructed to penalize titles which contained redundancies and excess information.\n"
        Instruction = Instruction + "1. Read the title and the source document carefully." + "\n"
        Instruction = Instruction + "2. Compare the title to the source document and identify the main points of the science paper." + "\n"
        Instruction = Instruction + "3. Assess how well the title covers the main points of the article, and how much irrelevant or redundant information it contains." + "\n"
        Instruction = Instruction + "4. Assign a relevance score from 1 to 5." + "\n"
    
    elif dimension == "informativeness":
        Name = "Informativeness"
        Description = "Informativeness(1-5) - the amount of information, including concepts, key points and findings, from the title. Annotators were instructed to penalize titles which contained under-explored concepts or redundancies and excess information.\n"
        Instruction = Instruction + "1. Read the title and the source title carefully." + "\n"
        Instruction = Instruction + "2. Check if the title effectively communicate the main topic, key points, and significant findings or conclusions. Assess if the information provided is comprehensive, accurate, and complete." + "\n"
        Instruction = Instruction + "3. A good informative title (scored 4 or 5) should convey enough amount of conpcets and key points in a progressive manner, while the relatively poorer ones have vague descriptions and under-explored ideas(scored 1 or 2)." + "\n"
        Instruction = Instruction + "4. Assign a score for informativeness on a scale of 1 to 5, to make the reuslts more clearly distributed, please avoid giving score 4" + "\n"
        
    elif dimension == "novelty":
        Name = "Novelty (1-3)"
        Description = "Novelty (1-3): compare the given title and previous reference studies, to measure the uniqueness and originality of the title in terms of concept, perspective, and creativity.  \n"
        Description = Description + "- 1: Poor. The title lacks originality and uniqueness, and it closely resembles existing or common titles. \n"
        Description = Description + "- 2: Fair. The title shows some degree of uniqueness and creativity, but it still has elements that are similar to references or predictable.\n"
        Description = Description + "- 3: Good. The title is highly original and unique, offering a fresh and creative perspective that stands out from common titles.\n"
        Instruction = "Assign a score for novelty on a scale of 1 to 3"

    prompt_str = ""
    if ICL != "":
        prompt_str = prompt_str + "You will be given one title written for a science paper and several reference papers.\nYour task is to rate the title on one metric.\n"
    else:
        prompt_str = prompt_str + "You will be given one title written for a science paper.\n"
        prompt_str = prompt_str + "Your task is to rate the output on one metric.\n"
    prompt_str = prompt_str + "Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.\n"
    prompt_str = prompt_str + "Evaluation Criteria:\n"
    prompt_str = prompt_str + Description
    prompt_str = prompt_str + Instruction
    if dimension not in dimensions_single_inputs:
        prompt_str = prompt_str + "Main topic:\n"
        prompt_str = prompt_str + source + "\n"
    if ICL != "":
        prompt_str = prompt_str + "Let's say there are some scientific paper novelty scores examples, each of which consists of a title, a group of references and a novelty score. \n"
        prompt_str = prompt_str + ICL
    if len(refs_texts)>0:
        prompt_str = prompt_str + "Please refer to the examples and assign a novelty score to the given title:\n"
        prompt_str = prompt_str + "\n".join(refs_texts) + "\n"
        
    prompt_str = prompt_str + "Predicted title:\n"
    prompt_str = prompt_str + output + "\n"
    prompt_str = prompt_str + "Evaluation Form (scores ONLY):\n"
    prompt_str = prompt_str + "- "+Name+":"

    return prompt_str

dimensions = ["novelty"]
geval_result = {}


i = 0
for j,id in enumerate(id2citations):
    geval_result[id] = {}
    if True:
        if id in papers_sampled and id in id2result_multi_agent:
            for dimension in dimensions:
                prompt = construct_prompt_g_eval(papers_sampled[id]["title"],id2result_multi_agent[id],id2citations[id],dimension)
                # print(prompt)
                
                # prompt = construct_prompt_g_eval(papers_sampled[id]["title"],papers_sampled[id]["abstract"],dimension)
                # break
                result = run_openai(prompt,key_bundles,i,i)
                # print(result)
                print(str(j),str(dimension),str(result))
                geval_result[id][dimension] = result
                i = i + 1
        # break
        
geval_avg_scores = {"informativeness":0,"novelty":0}
count = 0
for paperId in geval_result:
    if "novelty" in geval_result[paperId]:
        count = count + 1
        for dimension in dimensions:
            if "1. Fluency (1-3): " in geval_result[paperId][dimension]:
                geval_result[paperId][dimension].replace("geval_result[paperId][dimension]","")
            score = int(geval_result[paperId][dimension][0])
            geval_avg_scores[dimension] = geval_avg_scores[dimension] + score

for d in geval_avg_scores:
    print(d+ "\t" + str(geval_avg_scores[d]/count))
count

In [63]:
author_roles = pd.read_csv('../naive_method_v1.0/paper_author_roles.tsv', sep='\t', header=None, names=["paper_index","author_index","role"])

id2authorRole = {}
for i in range(len(author_roles["paper_index"])):
    paperId = author_roles["paper_index"][i]
    authorId = author_roles["author_index"][i]
    if paperId not in id2authorRole:
        id2authorRole[paperId] = {}
    id2authorRole[paperId][authorId] = author_roles["role"][i]
id2authorRole

{'130862d54894966552cb85d3ee6f739f885d4989': {40350773: 'Author',
  1705375: 'Author and Leading Researcher',
  1690158: 'Author and Leading Researcher'},
 'e28bdc373de80d7ec0e64631a89e64fbdcdae230': {1737649: 'The role of this author in paper writing could be a literature search specialist or a researcher specializing in search strategies and methodologies.',
  145278906: 'The role of this author in paper writing could be a co-author or contributor in the section discussing the importance of qualitative methods in software engineering research and providing guidelines for their use.',
  6648358: 'The role of this author in paper writing could be focused on the strategy and processes of software product customization in large-scale industrial settings. They could contribute to the discussion and analysis of strategies and processes used in the development of customized software products in industrial settings, particularly in the context of telecom companies in Germany and Sweden. They